In [68]:
import pandas as pd
import numpy as np

df = pd.read_csv("Public Parking Lots in the Loop.csv")

In [8]:
pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------- ----------------- 71.7/125.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [69]:
df.head(10)

,Parking Lot,Address,Spots,EV Chargers,Number of EV Chargers,Type of EV Charger,Hours,Days
0,AON Center Parking,"386 E. Lake St.\nChicago, IL 60601",678,NaN,NaN,NaN,24,7
1,Swissotel Chicago,"323 E Wacker Drive\nChicago, IL 60601",180,NaN,NaN,NaN,24,7
2,Village Market,"333 East Benton\nChicago, IL 60601",155,NaN,NaN,NaN,24,7
3,400 Randolph,"400 East Randolph,\nChicago, IL 60601",165,NaN,NaN,NaN,24,7
4,Millenium Park Garage,"6 South Columbus Dr.\nChicago, IL 60601",3800,NaN,NaN,NaN,24,7
5,Millenium Lakeside,"5 South Columbus Dr.\nChicago, IL 60601",3850,NaN,NaN,NaN,24,7
6,Grant Park South,"325 S. Michigan Ave.\nChicago, IL 60604",1300,NaN,NaN,NaN,24,7
7,Grant Park North,"25 North Michigan Ave., Chicago, IL, 60602",1850,NaN,NaN,NaN,24,7
8,60 East Lake Street,"60 East Lake Streetchicago, IL 60601",200,NaN,NaN,NaN,24,7
9,55 East Monroe,"55 East Adams Street, Chicago, 60603",703,NaN,NaN,NaN,24,7


In [70]:
from geopy.geocoders import OpenCage

# Replace 'YOUR_API_KEY' with your actual API key
api_key = '4ac4fa396f6741768becb60be068a3a5'

# Create a geolocator object
geolocator = OpenCage(api_key=api_key)

# Address to geocode
addresses = df['Address']

#print(addresses)
# Geocode the address
latitude = []
longitude = []
for a in addresses:
    location = geolocator.geocode(a)
    latitude.append(location.latitude)
    longitude.append(location.longitude)

    
#print(locations.latitude)
#geolocator.geocode(address)

In [71]:
df['latitude'] = latitude
df['longitude'] = longitude

In [72]:
def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

# Create a matrix to store distances
dist_matrix = np.zeros((len(df), len(df)))

# Calculate distance between each pair of locations
for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
            dist_matrix[i][j] = haversine(df.loc[i, 'latitude'], df.loc[i, 'longitude'],
                                          df.loc[j, 'latitude'], df.loc[j, 'longitude'])
        else:
            dist_matrix[i][j] = np.inf  # Set self-distance to infinity

print(dist_matrix)

[[       inf 0.70496274 0.7417462  0.9337431  2.10834178 0.69448072
  0.93064436 0.37765014 0.16337957 0.71909199 0.13847213 0.24538074
  0.14199413 0.46948637 0.46948637 0.27566872 0.48178893 0.63962876
  0.57821297 0.86903659 0.85378519]
 [0.70496274        inf 0.24955026 0.36050943 2.19665558 0.73780523
  1.12184274 0.56820557 0.54675009 1.02278255 0.71145362 0.90409837
  0.84682587 0.31256775 0.31256775 0.97008667 1.18261823 1.33930653
  1.27258421 1.5303505  1.26695493]
 [0.7417462  0.24955026        inf 0.19200489 1.95664555 0.53952929
  0.92186985 0.47696847 0.57991176 0.86046849 0.70033844 0.88744174
  0.87474081 0.27242768 0.27242768 1.01738849 1.21916759 1.37667869
  1.26165646 1.47886964 1.11832151]
 [0.9337431  0.36050943 0.19200489        inf 1.95970295 0.64728649
  1.01073103 0.65415566 0.77187531 0.98731701 0.88951701 1.07436195
  1.06633762 0.46435056 0.46435056 1.2093761  1.41066396 1.56804881
  1.44749706 1.65361949 1.24964021]
 [2.10834178 2.19665558 1.95664555 1.959

In [73]:
closest_locations = []

for i in range(len(df)):
    closest_index = np.argmin(dist_matrix[i])
    closest_location = df.loc[closest_index, 'Address']
    closest_locations.append(closest_location)

In [74]:
#df['closest_address'] = closest_locations
df.head(20)

,Parking Lot,Address,Spots,EV Chargers,Number of EV Chargers,Type of EV Charger,Hours,Days,latitude,longitude
0,AON Center Parking,"386 E. Lake St.\nChicago, IL 60601",678,NaN,NaN,NaN,24,7,41.885756,-87.627770
1,Swissotel Chicago,"323 E Wacker Drive\nChicago, IL 60601",180,NaN,NaN,NaN,24,7,41.887345,-87.619525
2,Village Market,"333 East Benton\nChicago, IL 60601",155,NaN,NaN,NaN,24,7,41.885159,-87.618845
3,400 Randolph,"400 East Randolph,\nChicago, IL 60601",165,NaN,NaN,NaN,24,7,41.884986,-87.616538
4,Millenium Park Garage,"6 South Columbus Dr.\nChicago, IL 60601",3800,NaN,NaN,NaN,24,7,41.867601,-87.620423
5,Millenium Lakeside,"5 South Columbus Dr.\nChicago, IL 60601",3850,NaN,NaN,NaN,24,7,41.881034,-87.622278
6,Grant Park South,"325 S. Michigan Ave.\nChicago, IL 60604",1300,NaN,NaN,NaN,24,7,41.877847,-87.624093
7,Grant Park North,"25 North Michigan Ave., Chicago, IL, 60602",1850,NaN,NaN,NaN,24,7,41.883620,-87.624223
8,60 East Lake Street,"60 East Lake Streetchicago, IL 60601",200,NaN,NaN,NaN,24,7,41.885803,-87.625797
9,55 East Monroe,"55 East Adams Street, Chicago, 60603",703,NaN,NaN,NaN,24,7,41.879447,-87.625859


In [50]:
df.to_csv("parking_spots_with_locations.csv")

In [57]:
df = pd.read_csv("parking_spots_with_locations.csv")

In [61]:
df.head(20)

,Unnamed: 0,Parking Lot,Address,Spots,EV Chargers,Number of EV Chargers,Type of EV Charger,Hours,Days,latitude,longitude,closest_address
0,0,AON Center Parking,"386 E. Lake St.\nChicago, IL 60601",678,NaN,NaN,NaN,24,7,41.885756,-87.627770,"20 East RandolphChicago, IL 60601"
1,1,Swissotel Chicago,"323 E Wacker Drive\nChicago, IL 60601",180,NaN,NaN,NaN,24,7,41.887345,-87.619525,"333 East Benton\nChicago, IL 60601"
2,2,Village Market,"333 East Benton\nChicago, IL 60601",155,NaN,NaN,NaN,24,7,41.885159,-87.618845,"400 East Randolph,\nChicago, IL 60601"
3,3,400 Randolph,"400 East Randolph,\nChicago, IL 60601",165,NaN,NaN,NaN,24,7,41.884986,-87.616538,"333 East Benton\nChicago, IL 60601"
4,4,Millenium Park Garage,"6 South Columbus Dr.\nChicago, IL 60601",3800,NaN,NaN,NaN,24,7,41.867601,-87.620423,"325 S. Michigan Ave.\nChicago, IL 60604"
5,5,Millenium Lakeside,"5 South Columbus Dr.\nChicago, IL 60601",3850,NaN,NaN,NaN,24,7,41.881034,-87.622278,"25 North Michigan Ave., Chicago, IL, 60602"
6,6,Grant Park South,"325 S. Michigan Ave.\nChicago, IL 60604",1300,NaN,NaN,NaN,24,7,41.877847,-87.624093,"55 East Adams Street, Chicago, 60603"
7,7,Grant Park North,"25 North Michigan Ave., Chicago, IL, 60602",1850,NaN,NaN,NaN,24,7,41.883620,-87.624223,"181 North ClarkChicago, IL 60601"
8,8,60 East Lake Street,"60 East Lake Streetchicago, IL 60601",200,NaN,NaN,NaN,24,7,41.885803,-87.625797,"386 E. Lake St.\nChicago, IL 60601"
9,9,55 East Monroe,"55 East Adams Street, Chicago, 60603",703,NaN,NaN,NaN,24,7,41.879447,-87.625859,"325 S. Michigan Ave.\nChicago, IL 60604"


In [62]:
df['EV Chargers'] = df['EV Chargers'].replace({'Yes': True}).fillna(False)

# Check the DataFrame
df.head(20)

C:\Users\ianre\AppData\Local\Temp\ipykernel_35800\2946355043.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['EV Chargers'] = df['EV Chargers'].replace({'Yes': True}).fillna(False)


,Unnamed: 0,Parking Lot,Address,Spots,EV Chargers,Number of EV Chargers,Type of EV Charger,Hours,Days,latitude,longitude,closest_address
0,0,AON Center Parking,"386 E. Lake St.\nChicago, IL 60601",678,False,NaN,NaN,24,7,41.885756,-87.627770,"20 East RandolphChicago, IL 60601"
1,1,Swissotel Chicago,"323 E Wacker Drive\nChicago, IL 60601",180,False,NaN,NaN,24,7,41.887345,-87.619525,"333 East Benton\nChicago, IL 60601"
2,2,Village Market,"333 East Benton\nChicago, IL 60601",155,False,NaN,NaN,24,7,41.885159,-87.618845,"400 East Randolph,\nChicago, IL 60601"
3,3,400 Randolph,"400 East Randolph,\nChicago, IL 60601",165,False,NaN,NaN,24,7,41.884986,-87.616538,"333 East Benton\nChicago, IL 60601"
4,4,Millenium Park Garage,"6 South Columbus Dr.\nChicago, IL 60601",3800,False,NaN,NaN,24,7,41.867601,-87.620423,"325 S. Michigan Ave.\nChicago, IL 60604"
5,5,Millenium Lakeside,"5 South Columbus Dr.\nChicago, IL 60601",3850,False,NaN,NaN,24,7,41.881034,-87.622278,"25 North Michigan Ave., Chicago, IL, 60602"
6,6,Grant Park South,"325 S. Michigan Ave.\nChicago, IL 60604",1300,False,NaN,NaN,24,7,41.877847,-87.624093,"55 East Adams Street, Chicago, 60603"
7,7,Grant Park North,"25 North Michigan Ave., Chicago, IL, 60602",1850,False,NaN,NaN,24,7,41.883620,-87.624223,"181 North ClarkChicago, IL 60601"
8,8,60 East Lake Street,"60 East Lake Streetchicago, IL 60601",200,False,NaN,NaN,24,7,41.885803,-87.625797,"386 E. Lake St.\nChicago, IL 60601"
9,9,55 East Monroe,"55 East Adams Street, Chicago, 60603",703,False,NaN,NaN,24,7,41.879447,-87.625859,"325 S. Michigan Ave.\nChicago, IL 60604"


In [66]:
import pandas as pd
import folium

# Create a Folium map centered around the average latitude and longitude
map_center = [df['latitude'].mean(), df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Add a marker for each location
for _, row in df.iterrows():
    color = 'green' if row['EV Chargers'] else 'blue'
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=row['Spots'] * 0.01,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(m)

# Save the map to an HTML file
m.save('map_with_parking_spots.html')